# LinkedIn Post Generator App

Here we will build an AI Application using Gemini, LangChain and Streamlit with the following features:

- Custom Landing Page
- LinkedIn Post Generation
- Streamlit features

## Install App and LLM dependencies

In [26]:
!pip install langchain==0.1.12 -q
!pip install langchain-google-genai==0.0.7 -q
!pip install langchain-community==0.0.29 -q
!pip install streamlit==1.32.2 -q
!pip install pyngrok==7.1.5 -q
!pip install google-generativeai>=0.3.2 -q

## Load Gemini API Credentials

Here we load it from a file so we don't explore the credentials on the internet by mistake

In [27]:
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')

In [28]:
%%writefile instagram_app.py

import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage

st.title("Instagram Post Generator")

# System prompt that defines the app's behavior
system_prompt = """
You are a creative Instagram post generator.
Your job is to write engaging Instagram captions or short posts based on the user's topic.


Guidelines:
- Keep the caption within 100-150 words
- Use emojis where appropriate (but not overused)
- Include 5-8 popular and relevant hashtags
- Maintain a casual, fun, and relatable tone
- Write like a real person, not a brand
- Add a question or CTA at the end if possible

The user will provide a topic or photo description. Your output should be formatted as a ready-to-post caption.
"""

# Initialize the LLM
gemini = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash-thinking-exp-01-21',
    temperature=0.8,
    convert_system_message_to_human=True
)

# Create the prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{user_input}")
])
# App description
st.markdown("### Create Instagram captions with a creative twist using AI")
st.markdown("Enter a topic or photo description to generate a post-worthy caption!")

# Input area with specific placeholder text
user_input = st.text_area("What's the post about?",
                          placeholder="Example: A sunset photo I clicked in Goa last weekend",
                          height=100)

# Generate button
if st.button("Generate Instagram Post"):
    if user_input:
        with st.spinner("Crafting your Insta caption..."):
            messages = prompt_template.format_messages(user_input=user_input)
            response = gemini.invoke(messages)

            st.markdown("### Your Instagram Caption:")
            st.markdown(response.content)
            st.markdown("---")
            st.markdown("Copy it, post it, and tag us if you love it! 📸")
    else:
        st.error("Please describe your post topic or photo to get started.")

Overwriting instagram_app.py


In [29]:
#from pyngrok import ngrok

# Terminate open tunnels if exist
#ngrok.kill()

#ngrok.set_auth_token(userdata.get('NGROK_API_KEY'))

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
#ngrok_tunnel = ngrok.connect(8989)
#print("Streamlit App:", ngrok_tunnel.public_url)

In [30]:
from pyngrok import ngrok
import os
from google.colab import userdata

# Kill any existing ngrok tunnels
ngrok.kill()

# Set NGROK auth token securely from your Colab secrets
ngrok.set_auth_token(userdata.get('NGROK_API_KEY'))

# Define the port Streamlit will run on
port = 8501

# Create HTTPS tunnel for Streamlit app
public_url = ngrok.connect(port, "http")
print(f"🌐 Streamlit App is live at: {public_url.public_url}")

# Save to environment so Streamlit knows which port to use
os.environ["STREAMLIT_SERVER_PORT"] = str(port)
os.environ["STREAMLIT_SERVER_HEADLESS"] = "true"


🌐 Streamlit App is live at: https://efa6-34-141-165-19.ngrok-free.app


In [31]:
!streamlit run instagram_app.py > /dev/null 2>&1 &


## Remove running app processes